In [ ]:
import sales_prediction.sales_prediction as sp
import torch
import torch.nn as nn
from sales_prediction.preproc_tabular import get_tabular
from sales_prediction.dataset_sales import getDataset
from transformers import AutoModel

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

img_path = 'C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\Dati\\ResizedImages\\'
tab_path = 'C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\Dati\\sales_anagrafica_final.csv'
aggregated_path = 'C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\Dati\\aggregated_sales_season.csv'
aggregated_month_path = 'C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\Dati\\aggregated_sales_month.csv'

model = torch.jit.load('C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\results\\agg_2\\weights.pt')
model.eval()
print("importato")
loss_fn = nn.MSELoss()
data, references, descriptions = get_tabular(aggregated_path)
train, val = getDataset(references, data, descriptions, img_path, 64)
avg_mse = 0
num_batches = len(val)

with torch.no_grad(): #si assicura che il gradiente qui non venga calcolato
        for img, tab, desc, y in val:
            img, tab, y = img.to(device), tab.to(device), y.to(device)
        
            bert = AutoModel.from_pretrained("distilbert-base-multilingual-cased")
            bert_res = bert(**desc)
            desc_tensor = bert_res.last_hidden_state[:,0,:]
            desc_tensor = desc_tensor.to(device)
        
            pred = model(img, tab, desc_tensor)
            avg_mse += loss_fn(pred.squeeze(), y.float()).item()
            print("calcolata loss")

avg_mse /= num_batches  
print('Validation Error: \n Avg MSE: {avg_mse:>8f}')
